In [19]:
import open3d as o3d
import numpy as np
import os
import gen_utils as gu
import pyvista as pv

In [41]:
challenge_path = '../datasets/3D_scans_per_patient_obj_files'
challenge_gt_path = '../datasets/ground-truth_labels_instances'
case = '01AS72AN'
jaw = 'lower'

osstem_mesh_path = './test_input_aligned/Case_01_lower(Antagonist)_1.obj'
# osstem_mesh_path = './test_input_aligned/Case_01_upper_1.obj'
# osstem_mesh_path = './test_input_aligned/Case_03_lower(Antagonist)_1.obj'
# osstem_mesh_path = './test_input_aligned/Case_03_upper_1.obj'

In [23]:
challenge_mesh_path = os.path.join(challenge_path, case, case+'_'+jaw+'.obj')
challenge_mesh_path

'../datasets/3D_scans_per_patient_obj_files/01AS72AN/01AS72AN_lower.obj'

# open3d.geometry.sample_points_poisson_disk

In [4]:
vertices, org_mesh = gu.read_txt_obj_ls(challenge_mesh_path, ret_mesh=True, use_tri_mesh=False)

gt_loaded_json = gu.load_json(os.path.join(challenge_gt_path, case, case+'_'+jaw+'.json'))
gt_labels = np.array(gt_loaded_json['labels']).reshape(-1)

if gt_loaded_json['jaw'] == 'lower':
    gt_labels -= 20
gt_labels[gt_labels//10==1] %= 10
gt_labels[gt_labels//10==2] = (gt_labels[gt_labels//10==2]%10) + 8
gt_labels[gt_labels<0] = 0

o3d.visualization.draw_geometries([org_mesh])

colored_mesh = gu.get_colored_mesh(org_mesh, gt_labels)

o3d.visualization.draw_geometries([colored_mesh])

In [5]:
np.array(colored_mesh.vertex_colors).dtype

dtype('float64')

In [6]:
pcd = colored_mesh.sample_points_poisson_disk(24000)

o3d.visualization.draw_geometries([pcd])

In [7]:
pcd.points

std::vector<Eigen::Vector3d> with 24000 elements.
Use numpy.asarray() to access data.

In [8]:
pcd.colors

std::vector<Eigen::Vector3d> with 24000 elements.
Use numpy.asarray() to access data.

In [9]:
color_ind = np.array(pcd.colors)
color_ind.dtype

dtype('float64')

In [10]:
palet = np.array([
        [255,153,153],

        [153,76,0],
        [153,153,0],
        [76,153,0],
        [0,153,153],
        [0,0,153],
        [153,0,153],
        [153,0,76],
        [64,64,64],

        [255,128,0],
        [153,153,0],
        [76,153,0],
        [0,153,153],
        [0,0,153],
        [153,0,153],
        [153,0,76],
        [64,64,64],
    ])/255
palet[9:] *= 0.4

# palet = np.array([
#         [0, 0, 0],
        
#         [10, 10, 10],
#         [20, 20, 20],
#         [30, 30, 30],
#         [40, 40, 40],
#         [50, 50, 50],
#         [60, 60, 60],
#         [70, 70, 70],
#         [80, 80, 80],
        
#         [90, 90, 90],
#         [100, 100, 100],
#         [110, 110, 110],
#         [120, 120, 120],
#         [130, 130, 130],
#         [140, 140, 140],
#         [150, 150, 150],
#         [160, 160, 160]
# ])

In [11]:
palet.dtype

dtype('float64')

In [12]:
sampled_label = np.zeros(24000)
for i, p in enumerate(palet):
    sampled_label[((np.isclose(color_ind, p)).sum(-1)//3).astype(np.bool_)] = i

In [13]:
for i in range(17):
    print(len(sampled_label[sampled_label==i]))

20323
178
188
245
237
237
435
353
0
177
224
240
218
208
401
336
0


In [14]:
print(len(color_ind[sampled_label==-1]))
color_ind[sampled_label==-1].astype(np.float64)

0


array([], shape=(0, 3), dtype=float64)

In [15]:
vertices, org_mesh = gu.read_txt_obj_ls(mesh_path, ret_mesh=True, use_tri_mesh=False)

cloud = pv.PolyData(np.array(pcd.points))
mesh = cloud.delaunay_2d()

org_mesh.vertices = o3d.utility.Vector3dVector(np.array(mesh.points))
org_mesh.triangles = o3d.utility.Vector3iVector(np.array(mesh.regular_faces))
org_mesh.compute_vertex_normals()

gu.print_3d(gu.get_colored_mesh(org_mesh, sampled_label))

In [16]:
np.array(pcd.normals).shape

(24000, 3)

# open3d.geometry.voxel_down_sample

In [195]:
vertices, org_mesh = gu.read_txt_obj_ls(challenge_mesh_path, ret_mesh=True, use_tri_mesh=True)
vertices.shape

(113080, 6)

In [201]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(np.array(vertices[:, :3]))
o3d.visualization.draw_geometries([pcd])

In [197]:
sorted(pcd.compute_nearest_neighbor_distance())[:10], sorted(pcd.compute_nearest_neighbor_distance())[-10:]

([0.0011825935361760406,
  0.0011825935361760406,
  0.0016410329603897648,
  0.0016410329603897648,
  0.002311960429093609,
  0.002311960429093609,
  0.0026146584176523994,
  0.0026146584176523994,
  0.0027376453954646067,
  0.0027376453954646067],
 [4.460783523945331,
  4.460783528249087,
  4.493911321230188,
  4.493911321230188,
  4.493913914127489,
  4.512324044190266,
  4.512324044190266,
  4.512324095029833,
  4.5826826959758105,
  4.706616869999504])

In [206]:
voxel_size = 2
sorted(pcd.voxel_down_sample(voxel_size).compute_nearest_neighbor_distance())[:10], \
sorted(pcd.voxel_down_sample(voxel_size).compute_nearest_neighbor_distance())[-10:]

([0.1325430225523643,
  0.1325430225523643,
  0.23117065895160044,
  0.23117065895160044,
  0.23117161264842515,
  0.23117161264842515,
  0.2362196846013414,
  0.2362196846013414,
  0.30239776030088816,
  0.30239776030088816],
 [4.498834763592838,
  4.509722311768017,
  4.509722311768017,
  4.51003835855039,
  4.512324044190266,
  4.512324044190266,
  4.512324095029833,
  4.512666483372474,
  4.5826826959758105,
  4.706616869999504])

In [207]:
o3d.visualization.draw_geometries([pcd.voxel_down_sample(voxel_size)])

In [129]:
np.array(pcd.points)[:10],  \
np.array((pcd.voxel_down_sample(0.1).points))[:10]

(array([[  32.23250961,   24.49151039, -111.03425598],
        [  32.79378128,   24.41981888, -111.25597382],
        [  32.48640823,   24.59178162, -111.42739105],
        [  32.45804977,   24.29204369, -110.77379608],
        [  32.09769821,   24.63780975, -111.29608154],
        [  32.04803848,   24.30966949, -110.60123444],
        [  32.79964828,   24.11439323, -110.6783905 ],
        [  31.68505669,   24.62940979, -111.09718323],
        [  32.43758774,   23.97652054, -110.2582016 ],
        [  33.24604797,   24.25520706, -111.28716278]]),
 array([[  -9.42953491,  -29.64219856, -102.44727325],
        [  -9.69323063,  -29.69818878, -102.41346741],
        [  -9.93113232,  -29.51582909, -102.50177002],
        [   0.42994091,  -29.39485741, -102.46543884],
        [   0.21797167,  -29.37998962, -102.33530426],
        [  -0.26125661,  -29.088974  , -102.40115356],
        [ -15.63829136,   -2.15891051, -111.1860199 ],
        [ -15.49253941,   -2.33407664, -111.36514282],
        